# Remote database (Render)

Render was chosen to host our PostgreSQL database. Render provides us with a professional, persistent database accessible from any environment, ideal for storing the clean version of your data in the Formatted Zone, or for centralizing results from your Spark pipeline and querying them with SQL.


## Installation and environment configuration: Java, Spark and JDBC



In [ ]:
# Instalar Java
!apt-get install openjdk-11-jdk -y

# Descargar Spark 3.4.2 (válido a abril 2025)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz

# Descomprimir Spark
!tar -xzf spark-3.4.2-bin-hadoop3.tgz

# Instalar findspark
!pip install -q findspark

# Descargar driver JDBC PostgreSQL
!wget -q https://jdbc.postgresql.org/download/postgresql-42.6.0.jar -O /content/postgresql-42.6.0.jar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.26+4-1ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


## Configure SparkSession with the PostgreSQL driver

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession

jar_path = "/content/postgresql-42.6.0.jar"

spark = SparkSession.builder \
    .appName("PostgreSQL-Spark") \
    .config("spark.jars", jar_path) \
    .config("spark.driver.extraClassPath", jar_path) \
    .getOrCreate()

## Create a sample DataFrame in PySpark

Here, a test DataFrame is created in PySpark with three games, including their game ID, title, and platform. This simulates data like what we would use in a formatted or analysis data area.

In [ ]:
data = [
    (1, "The Witcher 3", "PS4"),
    (2, "Elden Ring", "PS5"),
    (3, "God of War Ragnarok", "PS5")
]

columns = ["gameid", "title", "platform"]

df = spark.createDataFrame(data, columns)
df.show()

+------+-------------------+--------+
|gameid|              title|platform|
+------+-------------------+--------+
|     1|      The Witcher 3|     PS4|
|     2|         Elden Ring|     PS5|
|     3|God of War Ragnarok|     PS5|
+------+-------------------+--------+



## Configure JDBC connection with PostgreSQL (Render)

This block defines the connection parameters to a PostgreSQL database hosted on Render

In [ ]:
url = "jdbc:postgresql://dpg-cvsoi1buibrs73ed8cqg-a.frankfurt-postgres.render.com:5432/pyspark_postgres"

properties = {
    "user": "pyspark_postgres_user",
    "password": "INutwRtwChxqke9FwgicCJVT1vRDTHfI",
    "driver": "org.postgresql.Driver"
}

## Saving the DataFrame in PostgreSQL

In [ ]:
df.write \
    .jdbc(url=url, table="games", mode="overwrite", properties=properties)

## Reading data from PostgreSQL to Spark

In [ ]:
df_read = spark.read.jdbc(url=url, table="games", properties=properties)
df_read.show()

+------+-------------------+--------+
|gameid|              title|platform|
+------+-------------------+--------+
|     2|         Elden Ring|     PS5|
|     3|God of War Ragnarok|     PS5|
|     1|      The Witcher 3|     PS4|
+------+-------------------+--------+



## Connect to PostgreSQL and drop the table with psycopg2

In [ ]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.1 MB/s eta 0:00:00


In [ ]:
import psycopg2

# Datos de conexión (usá los mismos que con Spark)
conn = psycopg2.connect(
    host="dpg-cvsoi1buibrs73ed8cqg-a.frankfurt-postgres.render.com",
    port=5432,
    database="pyspark_postgres",
    user="pyspark_postgres_user",
    password="INutwRtwChxqke9FwgicCJVT1vRDTHfI"
)

cur = conn.cursor()

# Eliminar la tabla
cur.execute("DROP TABLE IF EXISTS games")

conn.commit()
cur.close()
conn.close()

print("✅ Tabla 'games' eliminada con éxito.")


✅ Tabla 'games' eliminada con éxito.
